# Universidad del Valle de Guatemala
## Facultad de Ingeniería
### Departamento de Computación

---

# Proyecto 1: Establecimientos educativos de Guatemala
## Avances del proyecto 1. Limpieza de datos - Dataset: Establecimientos educativos de Guatemala

**Integrantes:**
- Diego Alexander Hernández Silvestre, 21270
- Linda Inés Jiménez Vides, 21169
- Mario Antonio Guerra Morales, 21008
- David Jonathan Aragon Vasquez, 21053

**Curso:** Data Science  
**Sección:** 10  
**Grupo** 4  

---

Guatemala, 25 de julio de 2024


In [3]:
# Importar las librerías necesarias
import pandas as pd
import numpy as np

In [4]:
# Cargar el conjunto de datos
filePath = 'UnifiedData/UnifiedData.csv'  # Reemplaza con la ruta real de tu archivo CSV
dataFrame = pd.read_csv(filePath)

In [5]:
# Mostrar las primeras filas del conjunto de datos
print("📝 Primeras filas del conjunto de datos:")
dataFrame.head()

📝 Primeras filas del conjunto de datos:


,CODIGO,DISTRITO,DEPARTAMENTO,MUNICIPIO,ESTABLECIMIENTO,DIRECCION,TELEFONO,SUPERVISOR,DIRECTOR,NIVEL,SECTOR,AREA,STATUS,MODALIDAD,JORNADA,PLAN,DEPARTAMENTAL
0,16-01-0138-46,16-031,ALTA VERAPAZ,COBAN,COLEGIO COBAN,KM.2 SALIDA A SAN JUAN CHAMELCO ZONA 8,77945104,MERCEDES JOSEFINA TORRES GALVEZ,GUSTAVO ADOLFO SIERRA POP,DIVERSIFICADO,PRIVADO,URBANA,ABIERTA,MONOLINGUE,MATUTINA,DIARIO(REGULAR),ALTA VERAPAZ
1,16-01-0139-46,16-031,ALTA VERAPAZ,COBAN,COLEGIO PARTICULAR MIXTO VERAPAZ,KM 209.5 ENTRADA A LA CIUDAD,77367402,MERCEDES JOSEFINA TORRES GALVEZ,GILMA DOLORES GUAY PAZ DE LEAL,DIVERSIFICADO,PRIVADO,URBANA,ABIERTA,MONOLINGUE,MATUTINA,DIARIO(REGULAR),ALTA VERAPAZ
2,16-01-0140-46,16-031,ALTA VERAPAZ,COBAN,"COLEGIO ""LA INMACULADA""",7A. AVENIDA 11-109 ZONA 6,78232301,MERCEDES JOSEFINA TORRES GALVEZ,VIRGINIA SOLANO SERRANO,DIVERSIFICADO,PRIVADO,URBANA,ABIERTA,MONOLINGUE,MATUTINA,DIARIO(REGULAR),ALTA VERAPAZ
3,16-01-0141-46,16-005,ALTA VERAPAZ,COBAN,ESCUELA NACIONAL DE CIENCIAS COMERCIALES,2A CALLE 11-10 ZONA 2,79514215,RUDY ADOLFO TOT OCH,HÉCTOR ROLANDO CHUN POOU,DIVERSIFICADO,OFICIAL,URBANA,ABIERTA,MONOLINGUE,MATUTINA,DIARIO(REGULAR),ALTA VERAPAZ
4,16-01-0142-46,16-005,ALTA VERAPAZ,COBAN,INSTITUTO NORMAL MIXTO DEL NORTE 'EMILIO ROSAL...,3A AVE 6-23 ZONA 11,79521468,RUDY ADOLFO TOT OCH,VICTOR HUGO DOMÍNGUEZ REYES,DIVERSIFICADO,OFICIAL,URBANA,ABIERTA,BILINGUE,VESPERTINA,DIARIO(REGULAR),ALTA VERAPAZ


In [6]:
# Obtener el número total de filas y columnas
numFilas, numColumnas = dataFrame.shape
print(f"\n🔢 Número total de filas (datos crudos): {numFilas}")
print(f"🔠 Número total de variables: {numColumnas}")


🔢 Número total de filas (datos crudos): 9330
🔠 Número total de variables: 17


In [7]:
# Listar los nombres de las variables
print("\n🏷️ Nombres de las variables en el conjunto de datos:")
dataFrame.columns.tolist()


🏷️ Nombres de las variables en el conjunto de datos:


['CODIGO',
 'DISTRITO',
 'DEPARTAMENTO',
 'MUNICIPIO',
 'ESTABLECIMIENTO',
 'DIRECCION',
 'TELEFONO',
 'SUPERVISOR',
 'DIRECTOR',
 'NIVEL',
 'SECTOR',
 'AREA',
 'STATUS',
 'MODALIDAD',
 'JORNADA',
 'PLAN',
 'DEPARTAMENTAL']

In [8]:
# Obtener una estadística descriptiva general
descripcion = dataFrame.describe(include='all')

# Mostrar las estadísticas descriptivas generales en una tabla
print("📊 Estadísticas descriptivas generales:")
descripcion

📊 Estadísticas descriptivas generales:


,CODIGO,DISTRITO,DEPARTAMENTO,MUNICIPIO,ESTABLECIMIENTO,DIRECCION,TELEFONO,SUPERVISOR,DIRECTOR,NIVEL,SECTOR,AREA,STATUS,MODALIDAD,JORNADA,PLAN,DEPARTAMENTAL
count,9330,9122,9330,9330,9330,9279,8791,9121,8433,9330,9330,9330,9330,9330,9330,9330,9330
unique,9330,690,23,349,5616,6084,5464,659,4706,1,4,3,4,2,6,13,26
top,16-01-0138-46,01-403,CIUDAD CAPITAL,ZONA 1,INSTITUTO NACIONAL DE EDUCACION DIVERSIFICADA,CABECERA MUNICIPAL,22067425,CARLOS HUMBERTO GONZALEZ DE LEON,---,DIVERSIFICADO,PRIVADO,URBANA,ABIERTA,MONOLINGUE,DOBLE,DIARIO(REGULAR),GUATEMALA NORTE
freq,1,268,1564,633,330,268,21,333,83,9330,7955,7604,6545,9038,3036,5720,1050


In [9]:
# Candidatos a variable objetivo
candidatos = ['STATUS', 'MODALIDAD', 'JORNADA']

for variable in candidatos:
    print(f"🔍 Distribución de clases en la variable '{variable}':")
    distribucionClases = dataFrame[variable].value_counts()
    print(distribucionClases)
    porcentajeClases = distribucionClases / len(dataFrame) * 100
    print(f"\n📊 Porcentaje de cada clase en '{variable}':")
    print(porcentajeClases)
    print("\n" + "="*50 + "\n")

🔍 Distribución de clases en la variable 'STATUS':
STATUS
ABIERTA                  6545
CERRADA TEMPORALMENTE    2663
TEMPORAL TITULOS          119
TEMPORAL NOMBRAMIENTO       3
Name: count, dtype: int64

📊 Porcentaje de cada clase en 'STATUS':
STATUS
ABIERTA                  70.150054
CERRADA TEMPORALMENTE    28.542337
TEMPORAL TITULOS          1.275456
TEMPORAL NOMBRAMIENTO     0.032154
Name: count, dtype: float64


🔍 Distribución de clases en la variable 'MODALIDAD':
MODALIDAD
MONOLINGUE    9038
BILINGUE       292
Name: count, dtype: int64

📊 Porcentaje de cada clase en 'MODALIDAD':
MODALIDAD
MONOLINGUE    96.870311
BILINGUE       3.129689
Name: count, dtype: float64


🔍 Distribución de clases en la variable 'JORNADA':
JORNADA
DOBLE          3036
VESPERTINA     2567
MATUTINA       2382
SIN JORNADA     964
NOCTURNA        289
INTERMEDIA       92
Name: count, dtype: int64

📊 Porcentaje de cada clase en 'JORNADA':
JORNADA
DOBLE          32.540193
VESPERTINA     27.513398
MATUTINA       

In [10]:
# Listar todas las variables del conjunto de datos
variables = dataFrame.columns.tolist()

for variable in variables:
    # Contar el número de valores únicos en la variable
    num_valores_unicos = dataFrame[variable].nunique()
    
    # Imprimir el número de valores únicos
    print(f"🔍 Número de valores únicos en la variable '{variable}': {num_valores_unicos}")
    
    # Distribución de clases
    distribucionClases = dataFrame[variable].value_counts()
    print(f"\n🔍 Distribución de clases en la variable '{variable}':")
    print(distribucionClases)
    
    # Calcular el porcentaje de cada clase
    porcentajeClases = distribucionClases / len(dataFrame) * 100
    print(f"\n📊 Porcentaje de cada clase en '{variable}':")
    print(porcentajeClases)
    
    # Imprimir una línea separadora
    print("\n" + "="*50 + "\n")


🔍 Número de valores únicos en la variable 'CODIGO': 9330

🔍 Distribución de clases en la variable 'CODIGO':
CODIGO
16-01-0138-46    1
17-01-3993-46    1
17-03-0094-46    1
17-03-0095-46    1
17-03-0098-46    1
                ..
05-06-0129-46    1
05-06-0130-46    1
05-06-0132-46    1
05-06-0134-46    1
19-11-0018-46    1
Name: count, Length: 9330, dtype: int64

📊 Porcentaje de cada clase en 'CODIGO':
CODIGO
16-01-0138-46    0.010718
17-01-3993-46    0.010718
17-03-0094-46    0.010718
17-03-0095-46    0.010718
17-03-0098-46    0.010718
                   ...   
05-06-0129-46    0.010718
05-06-0130-46    0.010718
05-06-0132-46    0.010718
05-06-0134-46    0.010718
19-11-0018-46    0.010718
Name: count, Length: 9330, dtype: float64


🔍 Número de valores únicos en la variable 'DISTRITO': 690

🔍 Distribución de clases en la variable 'DISTRITO':
DISTRITO
01-403    268
11-017    176
05-033    167
01-411    167
18-008    130
         ... 
09-014      1
14-003      1
14-055      1
14-009      

### Variables encargadas - Mario Guerra

In [14]:
variables = ['ESTABLECIMIENTO', 'DIRECCION', 'TELEFONO', 'SUPERVISOR']
distribucion_establecimientos = ['INSTITUTO', 'ESCUELA', 'COLEGIO', 'LICEO', 'CENTRO']

def revision_datos(df, columnas):
    for col in columnas:
        print(f"Análisis preliminar de datos crudos. Columna: {col}.")

        valores_totales = df[col].count()
        print(f"Valores totales: {valores_totales}")

        valores_unicos = df[col].nunique()
        print(f"Valores únicos: {valores_unicos}")

        valores_nulos = df[col].isnull().sum()
        print(f"Valores nulos totales: {valores_nulos}")

        valores_sin_inconsistencias = df[col].str.strip().str.upper().dropna().unique()
        print(f"Valores sin inconsistencias obtenidos: {valores_sin_inconsistencias}")
        cant_valores_sin_inconsistencias = df[col].str.strip().str.upper().dropna().nunique()
        print(f"Cantidad de valores sin inconsistencias: {cant_valores_sin_inconsistencias}")

        if col == 'ESTABLECIMIENTO':
            # Normalizar los valores de la columna 'ESTABLECIMIENTO'
            df[col] = df[col].str.strip().str.upper()

            # Crear condiciones y elecciones para la distribución
            condiciones = [df[col].str.contains(establecimiento, na=False) for establecimiento in distribucion_establecimientos]
            elecciones = distribucion_establecimientos

            # Categorizar los establecimientos
            df['CATEGORIA_ESTABLECIMIENTO'] = np.select(condiciones, elecciones, default='OTRO')

            # Mostrar la distribución de las categorías
            distribucion = df['CATEGORIA_ESTABLECIMIENTO'].value_counts()
            print(f"Distribución de categorías de establecimientos:\n{distribucion}")
        
        if col == 'TELEFONO':
            # Contar valores que contienen un guion '-'
            telefonos_con_guion = df[col].str.contains('-').sum()
            print(f"Cantidad de valores en la columna TELEFONO que contienen un guion '-': {telefonos_con_guion}")

        
        print("\n")

        
revision_datos(dataFrame, variables)

Análisis preliminar de datos crudos. Columna: ESTABLECIMIENTO.
Valores totales: 9330
Valores únicos: 5616
Valores nulos totales: 0
Valores sin inconsistencias obtenidos: ['COLEGIO  COBAN' 'COLEGIO PARTICULAR MIXTO  VERAPAZ'
 'COLEGIO "LA INMACULADA"' ... 'LICEO PARTICULAR MIXTO "JIREH"'
 'LICEO PARTICULAR MIXTO  " JIREH"' 'INSTITUTO DIVERSIFICADO']
Cantidad de valores sin inconsistencias: 5616
Distribución de categorías de establecimientos:
CATEGORIA_ESTABLECIMIENTO
COLEGIO      3566
INSTITUTO    2649
LICEO        1531
CENTRO       1097
ESCUELA       311
OTRO          176
Name: count, dtype: int64


Análisis preliminar de datos crudos. Columna: DIRECCION.
Valores totales: 9279
Valores únicos: 6084
Valores nulos totales: 51
Valores sin inconsistencias obtenidos: ['KM.2 SALIDA A SAN JUAN CHAMELCO ZONA 8' 'KM 209.5 ENTRADA A LA CIUDAD'
 '7A. AVENIDA 11-109 ZONA 6' ... 'BARRIO EL AMATE' 'BARRIO  BUENOS  AIRES'
 'BARRIO EL  CAMPO']
Cantidad de valores sin inconsistencias: 6084


Análisis pr

### Variables encargadas - Diego Hernández 🔎

In [11]:
# Listado de las variables de interés
variables_interes = ['STATUS', 'MODALIDAD', 'JORNADA', 'PLAN', 'DEPARTAMENTAL']

# Función para realizar limpieza básica en columnas
def limpieza_basica(df, columnas):
    for columna in columnas:
        print(f"--- Análisis de la columna: {columna} ---")
        
        # Revisión de valores nulos
        nulos = df[columna].isnull().sum()
        print(f"🕵️‍♂️ Número de valores nulos en '{columna}': {nulos}")

        # Estadísticas generales
        print(f"🔢 Valores únicos en '{columna}': {df[columna].nunique()}")
        print(f"🔍 Primeros 5 valores en '{columna}': {df[columna].unique()[:5]}")
        
        # Detección de inconsistencias (espacios adicionales, capitalización)
        valores_limpios = df[columna].str.strip().str.upper().dropna().unique()
        print(f"🧹 Valores únicos después de limpieza en '{columna}': {valores_limpios}")

        # Distribución de clases
        distribucion_clases = df[columna].value_counts()
        print(f"📊 Distribución de clases en '{columna}':\n{distribucion_clases}")
        
        # Porcentaje de cada clase
        porcentaje_clases = distribucion_clases / len(df) * 100
        print(f"📈 Porcentaje de cada clase en '{columna}':\n{porcentaje_clases}")
        
        print("\n" + "="*50 + "\n")

# Ejecutar la función para las variables de interés
limpieza_basica(dataFrame, variables_interes)


--- Análisis de la columna: STATUS ---
🕵️‍♂️ Número de valores nulos en 'STATUS': 0
🔢 Valores únicos en 'STATUS': 4
🔍 Primeros 5 valores en 'STATUS': ['ABIERTA' 'CERRADA TEMPORALMENTE' 'TEMPORAL NOMBRAMIENTO'
 'TEMPORAL TITULOS']
🧹 Valores únicos después de limpieza en 'STATUS': ['ABIERTA' 'CERRADA TEMPORALMENTE' 'TEMPORAL NOMBRAMIENTO'
 'TEMPORAL TITULOS']
📊 Distribución de clases en 'STATUS':
STATUS
ABIERTA                  6545
CERRADA TEMPORALMENTE    2663
TEMPORAL TITULOS          119
TEMPORAL NOMBRAMIENTO       3
Name: count, dtype: int64
📈 Porcentaje de cada clase en 'STATUS':
STATUS
ABIERTA                  70.150054
CERRADA TEMPORALMENTE    28.542337
TEMPORAL TITULOS          1.275456
TEMPORAL NOMBRAMIENTO     0.032154
Name: count, dtype: float64


--- Análisis de la columna: MODALIDAD ---
🕵️‍♂️ Número de valores nulos en 'MODALIDAD': 0
🔢 Valores únicos en 'MODALIDAD': 2
🔍 Primeros 5 valores en 'MODALIDAD': ['MONOLINGUE' 'BILINGUE']
🧹 Valores únicos después de limpieza en 'MOD

### Variables encargadas - Linda Jiménez 🔎

In [12]:
# Listado de las variables de interés
variables_interes = ['DIRECTOR', 'NIVEL', 'SECTOR', 'AREA']

# Función para realizar limpieza básica en columnas
def limpieza_basica(df, columnas):
    for columna in columnas:
        print(f"--- Análisis de la columna: {columna} ---")
        
        # Revisión de valores nulos
        nulos = df[columna].isnull().sum()
        print(f"🕵️‍♂️ Número de valores nulos en '{columna}': {nulos}")

        # Estadísticas generales
        print(f"🔢 Valores únicos en '{columna}': {df[columna].nunique()}")
        print(f"🔍 Primeros 5 valores en '{columna}': {df[columna].unique()[:5]}")
        
        # Detección de inconsistencias (espacios adicionales, capitalización)
        valores_limpios = df[columna].str.strip().str.upper().dropna().unique()
        print(f"🧹 Valores únicos después de limpieza en '{columna}': {valores_limpios}")

        # Distribución de clases
        distribucion_clases = df[columna].value_counts()
        print(f"📊 Distribución de clases en '{columna}':\n{distribucion_clases}")
        
        # Porcentaje de cada clase
        porcentaje_clases = distribucion_clases / len(df) * 100
        print(f"📈 Porcentaje de cada clase en '{columna}':\n{porcentaje_clases}")
        
        print("\n" + "="*50 + "\n")

# Ejecutar la función para las variables de interés
limpieza_basica(dataFrame, variables_interes)

--- Análisis de la columna: DIRECTOR ---
🕵️‍♂️ Número de valores nulos en 'DIRECTOR': 897
🔢 Valores únicos en 'DIRECTOR': 4706
🔍 Primeros 5 valores en 'DIRECTOR': ['GUSTAVO ADOLFO SIERRA POP' 'GILMA DOLORES GUAY PAZ DE LEAL'
 'VIRGINIA  SOLANO SERRANO' 'HÉCTOR ROLANDO CHUN POOU'
 'VICTOR HUGO DOMÍNGUEZ REYES']
🧹 Valores únicos después de limpieza en 'DIRECTOR': ['GUSTAVO ADOLFO SIERRA POP' 'GILMA DOLORES GUAY PAZ DE LEAL'
 'VIRGINIA  SOLANO SERRANO' ... 'ANA MARÍA CUELLAR GUERRA'
 'MARLON JOSUÉ ARCHILA LORENZO' 'ROBIDIO  PORTILLO SALGUERO']
📊 Distribución de clases en 'DIRECTOR':
DIRECTOR
---                                83
----                               28
--                                 27
SIN DATO                           14
-----                              14
                                   ..
YENIFER NECTALÍ JERÓNIMO GARCÍA     1
BRENDA YANNET CRUZ ORREGO           1
NERY JONÁS MIRANDA HERNÁNDEZ        1
AURA VIRGINIA VASQUEZ RODRIGUEZ     1
ROBIDIO  PORTILLO SALGUE

In [13]:
# Importar las librerías necesarias
import pandas as pd

# Cargar el conjunto de datos
filePath = 'UnifiedData/UnifiedData.csv'  # Reemplaza con la ruta real de tu archivo CSV
dataFrame = pd.read_csv(filePath)

# Mostrar las primeras filas del conjunto de datos
print("📝 Primeras filas del conjunto de datos:")
print(dataFrame.head())

# Obtener el número total de filas y columnas
numFilas, numColumnas = dataFrame.shape
print(f"\n🔢 Número total de filas (datos crudos): {numFilas}")
print(f"🔠 Número total de variables: {numColumnas}")

# Listar los nombres de las variables
print("\n🏷️ Nombres de las variables en el conjunto de datos:")
print(dataFrame.columns.tolist())

# Verificación de consistencia
dataFrame['DISTRITO'] = dataFrame['DISTRITO'].str.upper()
dataFrame['DEPARTAMENTO'] = dataFrame['DEPARTAMENTO'].str.upper()
dataFrame['MUNICIPIO'] = dataFrame['MUNICIPIO'].str.upper()

# Función para agrupar categorías poco frecuentes
def agrupar_categorias(serie, umbral=0.01):
    frecuencia = serie.value_counts(normalize=True)
    categorias_poco_frecuentes = frecuencia[frecuencia < umbral].index
    serie = serie.apply(lambda x: 'OTROS' if x in categorias_poco_frecuentes else x)
    return serie

# Agrupar categorías poco frecuentes
dataFrame['DISTRITO'] = agrupar_categorias(dataFrame['DISTRITO'])
dataFrame['DEPARTAMENTO'] = agrupar_categorias(dataFrame['DEPARTAMENTO'])
dataFrame['MUNICIPIO'] = agrupar_categorias(dataFrame['MUNICIPIO'])

# Codificación de variables categóricas
dataFrame = pd.get_dummies(dataFrame, columns=['DISTRITO', 'DEPARTAMENTO', 'MUNICIPIO'])

# Mostrar el resultado final
print("\n📝 Primeras filas del conjunto de datos después del tratamiento:")
print(dataFrame.head())
print(f"\n🔢 Número total de filas después del tratamiento: {dataFrame.shape[0]}")
print(f"🔠 Número total de variables después del tratamiento: {dataFrame.shape[1]}")


📝 Primeras filas del conjunto de datos:
          CODIGO DISTRITO  DEPARTAMENTO MUNICIPIO  \
0  16-01-0138-46   16-031  ALTA VERAPAZ     COBAN   
1  16-01-0139-46   16-031  ALTA VERAPAZ     COBAN   
2  16-01-0140-46   16-031  ALTA VERAPAZ     COBAN   
3  16-01-0141-46   16-005  ALTA VERAPAZ     COBAN   
4  16-01-0142-46   16-005  ALTA VERAPAZ     COBAN   

                                     ESTABLECIMIENTO  \
0                                     COLEGIO  COBAN   
1                  COLEGIO PARTICULAR MIXTO  VERAPAZ   
2                            COLEGIO "LA INMACULADA"   
3           ESCUELA NACIONAL DE CIENCIAS COMERCIALES   
4  INSTITUTO NORMAL MIXTO DEL NORTE 'EMILIO ROSAL...   

                                DIRECCION  TELEFONO  \
0  KM.2 SALIDA A SAN JUAN CHAMELCO ZONA 8  77945104   
1            KM 209.5 ENTRADA A LA CIUDAD  77367402   
2               7A. AVENIDA 11-109 ZONA 6  78232301   
3                   2A CALLE 11-10 ZONA 2  79514215   
4                     3A AVE 